# MIBPR Demo
このnotebookには、コールドスタートの問題が発生しやすいデータセットのパフォーマンスをどのように改善するかを示す。 下記の論文を参考に実装する．

BPR: https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf

In [ ]:
from mibpr import *
from vectorize_documents import *

In [ ]:
from scipy.sparse import csr_matrix, coo_matrix, lil_matrix, csc_matrix, load_npz, save_npz
import numpy as np

train_name = 'maindata_256996/all_train256996.npz'
test_name = 'maindata_256996/all_test256996.npz'
feature_name = 'maindata_256996/all_features_seikika256996.npy'
price_name = 'maindata_256996/all_price256996.npy'
text_name = 'maindata_256996/all_titleword_vector256996.npy'
des_name = 'maindata_256996/all_description256996.npy'
vocab = 'maindata_256996/tradesy_vocab.npy'
train = load_npz(train_name)
test = load_npz(test_name)
feats = np.load(feature_name, allow_pickle= True, encoding='latin1')
price = np.load(price_name, allow_pickle= True, encoding='latin1')
text = np.load(text_name, allow_pickle= True, encoding='latin1')
des = np.load(des_name, allow_pickle= True, encoding='latin1')

## BPRの実装

最初に各要因を加えずに購入データのみで実装を行う

BPRは、特定のユーザーに対して, アイテムの絶対スコアを予測する行列分解モデルや最近傍法とは対照的に、ユーザーに対して, アイテムの相対ランキングを直接最適化するためのアルゴリズム.
これは、行列分解モデルの過学習を防ぐことにも、使用できる． 行列分解モデルは理論的には、ユーザーとアイテムの相互作用行列を完全に再構築できる潜在表現を学習するのに強力だが、BPRを使用すると、ユーザーのアイテムの相対的なランキングを直接最適化できるため、アイテムとユーザーのペアごとに絶対値を再構築する必要がなくなります。

しかし， この場合のテストデータには相互作用のない項目（新商品やユーザの情報が少ない）が多数あるため、コールドスタート問題が起こり、テストデータでのパフォーマンスが低いこと予想される．

In [ ]:
bpr = MIBPR()
bpr.fit(train, test, epochs=80, lr=.1, verbose=0)
print("Train AUC: {}".format(bpr.auc_score(train)))
print("Test AUC: {}".format(bpr.auc_score(test)))

#実装した場合，　トレーニングデータでは高くなるが，　テストデータはAUCの値が低いことがわかる．

## MIBPRの実装
コールドスタートの問題を軽減するために、アイテムのコンテンツベースの機能を利用する．　この場合、視覚的情報とタイトル名の情報，　金額の情報，　文章，つまりアイテムの説明文の情報を利用する．　

MIBPRアルゴリズムは、上記のコンテンツ機能を用いて，　アイテムとユーザーの潜在的なコンテンツスペースを学習する．　BPRを使用した行列分解モデルに上記のパラメーターを導入して，　潜在的なユーザーコンテンツ空間を表す行列と、潜在的なアイテムコンテンツ空間へのコンテンツベクトルのマッピングを学習するための埋め込み行列を分解モデルに加える．　潜在的なユーザーのコンテンツベクトルと潜在的なアイテムのコンテンツベクトルの内積は、ユーザーとアイテムのペアのコンテンツ関連性予測を表す．　この予測は、潜在的なアイテムとユーザーの要因に加えてバイアスからの予測に追加され、ユーザーとアイテムの相互作用とアイテムのコンテンツの両方によって予測を提供できる。

In [ ]:
import time
mibpr = MIVBPR()
mibpr.fit(train, test, item_content_features=feats, item_money_features=price, item_text_features=text, item_des_features=None, epochs=80, lr=.005)

print("Train AUC: {}".format(mivbpr.auc_score(train)))
print("Test AUC: {}".format(mivbpr.auc_score(test)))
print(time.time())

### アイテムコンテンツの視覚化
PCAとTSNEを使用してコンテンツベースを視覚化してアイテムがどういう風に表現される示す．

In [ ]:
import plotly
import plotly.graph_objs as go
from sklearn.manifold import TSNE

plotly.offline.init_notebook_mode(connected=True)

sample_indices = np.random.choice(len(text), 1000)
sample_embeddings = text[sample_indices]
tsne = TSNE()
tsne_embeddings = tsne.fit_transform(sample_embeddings)

In [ ]:
# 次元圧縮したベクトルを二次元で表示
item_text = [', '.join(i['tags']) if i is not None else '' for i in vocab]
trace = go.Scatter(
    x = tsne_embeddings[:,0],
    y = tsne_embeddings[:,1],
    mode = 'markers',
    texts = item_text
)

data = [trace]

plotly.offline.iplot(data, filename='tsne_embeddings_tradesy')

In [ ]:
from sklearn.decomposition import PCA

sample_indices = np.random.choice(len(text), 1000)
sample_embeddings = text[sample_indices]
pca = PCA(n_components=10)
pca_embeddings = pca.fit_transform(sample_embeddings)
pca_tsne = TSNE(perplexity=50)
pca_tsne_embeddings = tsne.fit_transform(pca_embeddings)

# Plot the embeddings in 2D
item_text = [', '.join(i['tags']) if i is not None else '' for i in vocab]
trace = go.Scatter(
    x = pca_tsne_embeddings[:,0],
    y = pca_tsne_embeddings[:,1],
    mode = 'markers',
    texts = item_text
)

data = [trace]

plotly.offline.iplot(data, filename='pca_tsne_embeddings_tradesy')

### ユーザに提案されるアイテムを確認
いくつかのアイテムを見て、それらの最も近い隣接アイテムが潜在空間と潜在コンテンツ空間にあるかを見て、MIBPRモデルがアイテムの合理的な表現を持っているかどうかを理解する。

In [ ]:
import scipy as sp

def nearest_neighbors(model, vocab, item_id, n=10):
    if items[item_id] is not None:
        tags = ', '.join(vocab[item_id]['tags'])
    else:
        tags = ''
    item_latent = model.latent_items[item_id]
    distances = [(item_id, tags, 0)]
    for i in range(len(model.latent_items)):
        if i == item_id:
            continue
        if vocab[i] is not None:
            tags = ', '.join(vocab[i]['tags'])
        else:
            tags = ''
        dist = sp.spatial.distance.cosine(item_latent, model.latent_items[i])
        distances.append((i, tags, dist))
    return sorted(distances, key=lambda x: x[2])[:n]
                         
def nearest_neighbors_content(model, vocab, item_id, n=10):
    if items[item_id] is not None:
        tags = ', '.join(vocab[item_id]['tags'])
    else:
        tags = ''
    item_content_latent = model.text_embedding_matrix @ model.item_text_features[item_id]
    distances = [(item_id, tags, 0)]
    for i in range(len(model.latent_items)):
        if i == item_id:
            continue
        if items[i] is not None:
            tags = ', '.join(items[i]['tags'])
        else:
            tags = ''
        other_item = model.text_embedding_matrix @ model.item_text_features[i]
        dist = sp.spatial.distance.cosine(item_text_latent, other_item)
        distances.append((i, tags, dist))
    return sorted(distances, key=lambda x: x[2])[:n]

print("By latent representations:")
nearest = nearest_neighbors(mibpr, items, 15224)
for neighbor in nearest:
    print(neighbor)
print()
print("By latent content representations:")
nearest_by_content = nearest_neighbors_content(mibpr, items, 15224)
for neighbor in nearest_by_content:
    print(neighbor)
print()
